In [61]:
import os
import json
import pandas as pd
import traceback

In [62]:
from langchain_openai import ChatOpenAI

In [63]:
from dotenv import load_dotenv

load_dotenv()

True

In [64]:
KEY=os.getenv("OPENAI_API_KEY")

In [65]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [66]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000232D28676D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000232D2867D60>, root_client=<openai.OpenAI object at 0x00000232D2867EE0>, root_async_client=<openai.AsyncOpenAI object at 0x00000232D2867970>, temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [67]:
#  New imports for OpenAI (moved to langchain_openai)
from langchain_openai import OpenAI, ChatOpenAI

#  Prompt template (still in langchain)
from langchain.prompts import PromptTemplate

#  Chains (still in langchain)
from langchain.chains import LLMChain, SequentialChain

#  Callbacks for token usage and cost tracking
from langchain.callbacks import get_openai_callback

#  PDF reading library
import PyPDF2


In [68]:
RESPONSE_JSON = {
    "1": {
        "mcq": "Write your first multiple choice question here.",
        "options": {
            "a": "First option text.",
            "b": "Second option text.",
            "c": "Third option text.",
            "d": "Fourth option text."
        },
        "correct": "a"
    },
    "2": {
        "mcq": "Write your second multiple choice question here.",
        "options": {
            "a": "First option text.",
            "b": "Second option text.",
            "c": "Third option text.",
            "d": "Fourth option text."
        },
        "correct": "b"
    },
    "3": {
        "mcq": "Write your third multiple choice question here.",
        "options": {
            "a": "First option text.",
            "b": "Second option text.",
            "c": "Third option text.",
            "d": "Fourth option text."
        },
        "correct": "d"
    }
}


In [69]:
TEMPLATE = """
You are an expert MCQ generator.

Given the following text, create exactly {number} multiple-choice questions for {subject} students, written in a {tone} tone.

**Input Text:**
{text}

**Important Instructions:**
- Do not repeat any questions.
- Each question must be directly based on the input text.
- All questions should be clear, unambiguous, and appropriate for {subject} students.
- Your response **must** be formatted exactly as the JSON example shown below.
- **Do not include any explanations or additional text. Only return the JSON object.**

**Response JSON Example:**
{response_json}

**Reminder:**
Ensure the JSON includes exactly {number} unique MCQs derived from the provided text.
"""


In [70]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [71]:
quiz_chain = quiz_generation_prompt | llm

In [72]:
TEMPLATE2 = """
You are an expert English grammarian and academic writer.

Given the following multiple-choice quiz intended for {subject} students:

### Quiz MCQs:
{quiz}

**Instructions:**
1. Evaluate the complexity of the quiz and provide a short analysis (maximum 50 words) describing whether the questions are appropriate for the cognitive and analytical abilities of the students.
2. If any questions are too difficult, too simple, or inappropriate, rewrite only those questions so they are better suited to the target students. Use a clear and consistent tone appropriate for {subject} students.
3. If no changes are needed, simply state "All questions are appropriate."

**Response Format:**
Return your answer as a JSON object with the following fields:

{{
  "complexity_analysis": "Your 50-word analysis here.",
  "updated_quiz": "The updated quiz here if any questions were changed, otherwise write 'No changes needed.'"
}}
"""

In [73]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [74]:
review_chain = quiz_evaluation_prompt | llm

In [75]:
def generate_and_evaluate(text, number, subject, tone, response_json):
    quiz_result = quiz_chain.invoke({
        "text": text,
        "number": str(number),
        "subject": subject,
        "tone": tone,
        "response_json": response_json
    })
    review_result = review_chain.invoke({
        "subject": subject,
        "quiz": quiz_result
    })
    return {
        "quiz": quiz_result,
        "review": review_result
    }

In [76]:
file_path=r"E:\promptquiz\data.txt"

In [77]:
file_path

'E:\\promptquiz\\data.txt'

In [78]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [79]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [80]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "Write your first multiple choice question here.", "options": {"a": "First option text.", "b": "Second option text.", "c": "Third option text.", "d": "Fourth option text."}, "correct": "a"}, "2": {"mcq": "Write your second multiple choice question here.", "options": {"a": "First option text.", "b": "Second option text.", "c": "Third option text.", "d": "Fourth option text."}, "correct": "b"}, "3": {"mcq": "Write your third multiple choice question here.", "options": {"a": "First option text.", "b": "Second option text.", "c": "Third option text.", "d": "Fourth option text."}, "correct": "d"}}'

In [81]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [82]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = generate_and_evaluate(
        text=TEXT,
        number=NUMBER,
        subject=SUBJECT,
        tone=TONE,
        response_json=json.dumps(RESPONSE_JSON)
    )

In [83]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: ${cb.total_cost:.4f}")

Total Tokens: 2023
Prompt Tokens: 1582
Completion Tokens: 441
Total Cost: $0.0015


In [84]:
response

{'quiz': AIMessage(content='\n{\n    "1": {\n        "mcq": "What is the main theme that explains the unity and diversity of life in biology?",\n        "options": {\n            "a": "Energy processing",\n            "b": "Evolution",\n            "c": "Regulation of internal environments",\n            "d": "Cellular structure"\n        },\n        "correct": "b"\n    },\n    "2": {\n        "mcq": "What is the scientific study of life known as?",\n        "options": {\n            "a": "Chemistry",\n            "b": "Biology",\n            "c": "Physics",\n            "d": "Geology"\n        },\n        "correct": "b"\n    },\n    "3": {\n        "mcq": "What is the process that allows organisms to move, grow, and reproduce?",\n        "options": {\n            "a": "Evolution",\n            "b": "Energy processing",\n            "c": "Cellular respiration",\n            "d": "Photosynthesis"\n        },\n        "correct": "b"\n    },\n    "4": {\n        "mcq": "At what levels of 

In [85]:
quiz=response.get("quiz")

In [87]:
quiz = json.loads(quiz.content)

In [88]:
quiz

{'1': {'mcq': 'What is the main theme that explains the unity and diversity of life in biology?',
  'options': {'a': 'Energy processing',
   'b': 'Evolution',
   'c': 'Regulation of internal environments',
   'd': 'Cellular structure'},
  'correct': 'b'},
 '2': {'mcq': 'What is the scientific study of life known as?',
  'options': {'a': 'Chemistry',
   'b': 'Biology',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'b'},
 '3': {'mcq': 'What is the process that allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Evolution',
   'b': 'Energy processing',
   'c': 'Cellular respiration',
   'd': 'Photosynthesis'},
  'correct': 'b'},
 '4': {'mcq': 'At what levels of organization can biologists study life?',
  'options': {'a': 'Only at the cellular level',
   'b': 'From molecular biology to the evolution of populations',
   'c': 'Only at the organism level',
   'd': 'From anatomy to physiology'},
  'correct': 'b'},
 '5': {'mcq': 'When did life on Earth emerge?',
  'option

In [89]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [90]:
quiz_table_data

[{'MCQ': 'What is the main theme that explains the unity and diversity of life in biology?',
  'Choices': 'a: Energy processing | b: Evolution | c: Regulation of internal environments | d: Cellular structure',
  'Correct': 'b'},
 {'MCQ': 'What is the scientific study of life known as?',
  'Choices': 'a: Chemistry | b: Biology | c: Physics | d: Geology',
  'Correct': 'b'},
 {'MCQ': 'What is the process that allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Evolution | b: Energy processing | c: Cellular respiration | d: Photosynthesis',
  'Correct': 'b'},
 {'MCQ': 'At what levels of organization can biologists study life?',
  'Choices': 'a: Only at the cellular level | b: From molecular biology to the evolution of populations | c: Only at the organism level | d: From anatomy to physiology',
  'Correct': 'b'},
 {'MCQ': 'When did life on Earth emerge?',
  'Choices': 'a: 1 million years ago | b: 3.7 billion years ago | c: 500 million years ago | d: 10,000 years ago',
  'Corr

In [91]:
quiz=pd.DataFrame(quiz_table_data)

In [92]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,What is the main theme that explains the unity...,a: Energy processing | b: Evolution | c: Regul...,b
1,What is the scientific study of life known as?,a: Chemistry | b: Biology | c: Physics | d: Ge...,b
2,What is the process that allows organisms to m...,a: Evolution | b: Energy processing | c: Cellu...,b
3,At what levels of organization can biologists ...,a: Only at the cellular level | b: From molecu...,b
4,When did life on Earth emerge?,a: 1 million years ago | b: 3.7 billion years ...,b


In [93]:
quiz.to_csv("machinelearning.csv", index=False)

In [94]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'07_11_2025_23_51_12'